In [2]:
import crandata
import xarray as xr
import pandas as pd
import numpy as np

In [5]:
X = xr.DataArray(np.arange(20).reshape(4, 5), dims=["obs", "var"])
obsm = {"embedding": xr.DataArray(np.random.rand(4, 2), dims=["obs", "other"])}
varm = {"feature": xr.DataArray(np.random.rand(5, 3), dims=["var", "other"])}
layers = {"layer1": X.copy()}
varp = {"contacts": xr.DataArray(np.random.rand(5, 5), dims=["var_0", "var_1"])}
obsp = {"adj": xr.DataArray(np.random.rand(4, 4), dims=["obs_0", "obs_1"])}
data = crandata.crandata.CrAnData(
    X, uns={"extra": "test"},
    obsm=obsm, varm=varm, layers=layers, varp=varp, obsp=obsp
)


In [6]:
data

CrAnData (primary: 'X') with axes: obs=4 (in-memory), var=5 (in-memory)
  X: [obs=4, var=5] (in-memory)
  obs: None
  var: None
  uns: {'extra': 'test'}
  obsm: {'embedding': <xarray.DataArray (obs: 4, other: 2)> Size: 64B
array([[0.22985535, 0.94814156],
       [0.64438283, 0.6637138 ],
       [0.28006699, 0.77574243],
       [0.48166066, 0.95720116]])
Dimensions without coordinates: obs, other}
  varm: {'feature': <xarray.DataArray (var: 5, other: 3)> Size: 120B
array([[0.01566936, 0.55565974, 0.49988154],
       [0.60603383, 0.11160247, 0.25562868],
       [0.63229154, 0.1040184 , 0.35389713],
       [0.68509591, 0.76481753, 0.00623495],
       [0.35098796, 0.68921427, 0.57629274]])
Dimensions without coordinates: var, other}
  layers: {'layer1': <xarray.DataArray (obs: 4, var: 5)> Size: 160B
array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])
Dimensions without coordinates: obs, var}
  varp: {'contacts': <xarray.DataA

In [8]:
import os
import tempfile
from pathlib import Path
import pandas as pd
import numpy as np
import pyBigWig
import importlib
import xarray as xr
import tqdm
import copy

import crandata
crandata = importlib.reload(crandata)

import crandata.chrom_io
crandata.crandata = importlib.reload(crandata.crandata)
crandata.chrom_io = importlib.reload(crandata.chrom_io)
crandata._anndatamodule = importlib.reload(crandata._anndatamodule)
crandata._dataloader = importlib.reload(crandata._dataloader)
from crandata._anndatamodule import MetaAnnDataModule

# Create temporary directories for beds, bigwigs, etc.
temp_dir = tempfile.TemporaryDirectory()
base_dir = Path(temp_dir.name)
beds_dir = base_dir / "beds"
bigwigs_dir = base_dir / "bigwigs"
beds_dir.mkdir(exist_ok=True)
bigwigs_dir.mkdir(exist_ok=True)

# Create a chromsizes file
chromsizes_file = base_dir / "chrom.sizes"
with open(chromsizes_file, "w") as f:
    f.write("chr1\t1000\n")

# Create two BED files (ClassA and ClassB)
bed_data_A = pd.DataFrame({0: ["chr1", "chr1"],
                           1: [100, 300],
                           2: [200, 400]})
bed_data_B = pd.DataFrame({0: ["chr1", "chr1"],
                           1: [150, 350],
                           2: [250, 450]})
bed_file_A = beds_dir / "ClassA.bed"
bed_file_B = beds_dir / "ClassB.bed"
bed_data_A.to_csv(bed_file_A, sep="\t", header=False, index=False)
bed_data_B.to_csv(bed_file_B, sep="\t", header=False, index=False)

# Create a consensus BED file
consensus = pd.DataFrame({0: ["chr1", "chr1", "chr1"],
                          1: [100, 300, 350],
                          2: [200, 400, 450]})
consensus_file = base_dir / "consensus.bed"
consensus.to_csv(consensus_file, sep="\t", header=False, index=False)

# Create a bigWig file with a single chromosome region
bigwig_file = bigwigs_dir / "test.bw"
bw = pyBigWig.open(str(bigwig_file), "w")
bw.addHeader([("chr1", 1000)])
bw.addEntries(chroms=["chr1"], starts=[0], ends=[1000], values=[5.0])
bw.close()

bigwig_file = bigwigs_dir / "test2.bw"
bw = pyBigWig.open(str(bigwig_file), "w")
bw.addHeader([("chr1", 1000)])
bw.addEntries(chroms=["chr1"], starts=[0], ends=[1000], values=[4.0])
bw.close()

# Set parameters for extraction
target_region_width = 100
backed_path = os.path.join(base_dir, "chrom_data.h5")

# Create the CrAnData object from the bigWig files and consensus regions
adata = crandata.chrom_io.import_bigwigs(
    bigwigs_folder=bigwigs_dir,
    regions_file=consensus_file,
    backed_path=backed_path,
    target_region_width=target_region_width,
    chromsizes_file=chromsizes_file,  
)

# Ensure obsm is a dictionary before adding entries.
if adata.obsm is None:
    adata.obsm = {}

# Add a random obsm entry
adata.obsm['gex'] = xr.DataArray(np.random.randn(adata.n_obs, 50),
                                 dims=['obs', 'genes'])

# Create a synthetic BEDP file for Hi-C contacts and add contacts to adata.varp
synthetic_bedp = pd.DataFrame({
    0: ["chr1", "chr1"],
    1: [100, 300],
    2: [200, 400],
    3: ["chr1", "chr1"],
    4: [150, 350],
    5: [250, 450],
    6: [10, 20]
})
synthetic_bedp_file = base_dir / "test2.bedp"
synthetic_bedp.to_csv(synthetic_bedp_file, sep="\t", header=False, index=False)

from crandata.chrom_io import add_contact_strengths_to_varp
contacts = add_contact_strengths_to_varp(adata, [str(synthetic_bedp_file)], key="hic_contacts")

print("Added Hi-C contact data to adata.varp['hic_contacts']:")
print("Shape:", adata.varp["hic_contacts"].shape)
print(adata.varp["hic_contacts"])

# Write to HDF5 and load back.
h5_path = os.path.join(base_dir, "adata.h5")
adata.to_h5(h5_path)
adata_loaded = crandata.crandata.CrAnData.from_h5(h5_path, backed=['X'])
print("\nDirectory contents:", os.listdir(base_dir))
print("\nLoaded CrAnData from HDF5:")
print(adata_loaded)
print("obs:")
print(adata_loaded.obs)
print("var:")
print(adata_loaded.var)
print("varp keys:", list(adata_loaded.varp.keys()))
if "hic_contacts" in adata_loaded.varp:
    print("Hi-C contact data shape:", adata_loaded.varp["hic_contacts"].shape)
    print(adata_loaded.varp["hic_contacts"])

# Create two copies of the loaded CrAnData (simulate two different datasets/species)
adata1 = copy.deepcopy(adata_loaded)
adata2 = copy.deepcopy(adata_loaded)
# Ensure each has a 'split' column
adata1.var["split"] = "train"
adata2.var["split"] = "train"

# Create a dummy FASTA file for the genome (with a single record for chr1)
fasta_file = base_dir / "chr1.fa"
with open(fasta_file, "w") as f:
    f.write(">chr1\n")
    f.write("A" * 1000 + "\n")

# Instead of passing a string, create a Genome object.
from crandata._genome import Genome
dummy_genome = Genome(str(fasta_file), chrom_sizes=str(chromsizes_file))

# Instantiate MetaAnnDataModule with the two datasets and corresponding genomes.
meta_module = crandata._anndatamodule.MetaAnnDataModule(
    adatas=[adata1, adata2],
    genomes=[dummy_genome, dummy_genome],
    data_sources={'y': 'X', 'hic': 'varp/hic_contacts', 'gex': 'obsm/gex'},
    in_memory=True,
    random_reverse_complement=True,
    max_stochastic_shift=5,
    deterministic_shift=False,
    shuffle_obs=True,
    shuffle=True,
    batch_size=3,    # small batch size for testing
    epoch_size=10    # small epoch size for quick testing
)

# Setup the meta module for the "fit" stage (train/val)
meta_module.setup("fit")

# Retrieve the training dataloader from the meta module and iterate over a couple of batches.
meta_train_dl = meta_module.train_dataloader

print("\nIterating over a couple of training batches from MetaAnnDataModule:")
for i, batch in enumerate(tqdm.tqdm(meta_train_dl.data)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
        # print(tensor)
    # For quick testing, you can uncomment the following to break early:
    # if i == 1:
    #     break

print("Final directory contents:", os.listdir(base_dir))

temp_dir.cleanup()


100%|██████████| 2/2 [00:00<00:00, 13819.78it/s]
2025-03-16 18:13:14.666 | INFO     | crandata.chrom_io:import_bigwigs:290 - Extracting values from 2 bigWig files...
2it [00:00, 102.74it/s]

Added Hi-C contact data to adata.varp['hic_contacts']:
Shape: (3, 3, 1)
<xarray.DataArray (var: 3, var_target: 3, obs: 1)> Size: 140B
<COO: shape=(3, 3, 1), dtype=float32, nnz=5, fill_value=0.0>
Coordinates:
  * var         (var) <U12 144B 'chr1:100-200' 'chr1:300-400' 'chr1:350-450'
  * var_target  (var_target) <U12 144B 'chr1:100-200' ... 'chr1:350-450'
  * obs         (obs) <U5 20B 'test2'

Directory contents: ['beds', 'bigwigs', 'chrom.sizes', 'consensus.bed', 'chrom_data.h5', 'test2.bedp', 'adata.h5']

Loaded CrAnData from HDF5:
CrAnData (primary: 'X') with axes: var=3 (backed), obs=2 (backed), seq_len=100 (backed)
  X: [var=3, obs=2, seq_len=100] (backed)
  obs: {'file_path': array(['/scratch/fast/46994/tmp7ojawwdz/bigwigs/test.bw',
       '/scratch/fast/46994/tmp7ojawwdz/bigwigs/test2.bw'], dtype='<U48'), 'obs': <xarray.DataArray (obs: 2)> Size: 16B
array([b'test', b'test2'], dtype=object)
Dimensions without coordinates: obs}
  var: {'chr': array(['chr1', 'chr1', 'chr1'], dtype=

AttributeError: 'float' object has no attribute 'sum'

In [10]:
adata.var['train_probs']

KeyError: 'train_probs'

In [5]:
adata._propagate_missing_coordinates()

In [6]:
adata

CrAnData (primary: 'X') with axes: var=3 (backed), obs=2 (backed), seq_len=100 (backed)
  X: [var=3, obs=2, seq_len=100] (backed)
  obs:                                               file_path
test    /scratch/fast/46987/tmpfckoy7a7/bigwigs/test.bw
test2  /scratch/fast/46987/tmpfckoy7a7/bigwigs/test2.bw
  var:              chrom  start  end  chunk_index   chr
region                                           
chr1:100-200  chr1    100  200            0  chr1
chr1:300-400  chr1    300  400            0  chr1
chr1:350-450  chr1    350  450            0  chr1
  uns: {'params': {'chunk_size': np.int64(512), 'max_stochastic_shift': np.int64(0), 'shifted_region_width': np.int64(100), 'target_region_width': np.int64(100)}}
  obsm: {'gex': <xarray.DataArray (obs: 2, genes: 50)> Size: 800B
array([[-0.84286505, -0.3330085 ,  2.1528912 , -0.45353179,  0.780963  ,
        -1.45020481, -1.88673658, -0.50893806,  0.45560893,  0.56886984,
         1.43031937, -1.16437629, -0.69579686, -0.39076824,  0.

In [ ]:
for i, batch in enumerate(tqdm.tqdm(meta_train_dl.data)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
        # print(tensor)
    # For quick testing, you can uncomment the following to break early:
    # if i == 1:
    #     break

print("Final directory contents:", os.listdir(base_dir))


In [ ]:
batch['sequence'].shape

In [ ]:
batch['hic'].shape

In [ ]:
adata.global_axis_order

In [ ]:
fff

In [ ]:
import cProfile

code = '''
for i, batch in enumerate(meta_train_dl.data):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
'''

cProfile.run(code)


In [ ]:
import crandata
import os
import crested
from tqdm import tqdm

In [ ]:
genomes = {}
beds = {}
chromsizes_files = {}
bed_files = {}
species = ['mouse','human','macaque']

WINDOW_SIZE = 2114
OFFSET = WINDOW_SIZE // 2  # e.g., 50% overlap
N_THRESHOLD = 0.3
n_bins = WINDOW_SIZE//50

In [ ]:
for s in species:
    genome_path = '/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/genome/onehots/'+s
    fasta_file = os.path.join(genome_path,s+'.fa')
    chrom_sizes = os.path.join(genome_path,s+'.fa.sizes')
    annotation_gtf_file = os.path.join(genome_path,s+'.annotation.gtf')
    chromsizes_files[s] = chrom_sizes
    genome = crandata.Genome(fasta_file, chrom_sizes, annotation_gtf_file)
    genomes[s] = genome
    # Set parameters for binning.
    
    # Optionally specify an output path for the BED file.
    OUTPUT_BED = os.path.join(genome_path, "binned_genome.bed")
    bed_files[s] = OUTPUT_BED
    # Generate bins and optionally write to disk.
    # binned_df = crandata.bin_genome(genome, WINDOW_SIZE, OFFSET, n_threshold=N_THRESHOLD, output_path=OUTPUT_BED)
    # print("Filtered bins:")
    # print(binned_df)

In [ ]:
adatas = {}

for s in species:
    # bigwigs_dir = os.path.join('/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/SpinalCord/manuscript/ATAC',s,'Group_bigwig')
    # adata = crandata.chrom_io.import_bigwigs(
    #     bigwigs_folder=bigwigs_dir,
    #     regions_file=bed_files[s],
    #     backed_path='/home/matthew.schmitz/Matthew/'+s+'_spc_test.h5',
    #     target_region_width=WINDOW_SIZE,
    #     chromsizes_file=chromsizes_files[s],
    #     target = 'mean',
    #     n_bins=n_bins
    # )
    # adatas[s] = adata
    adatas[s] = crandata.crandata.CrAnData.from_h5('/home/matthew.schmitz/Matthew/'+s+'_spc_test.h5')
    

In [ ]:
# import numpy as np
# adatas['mouse'].uns['chunk_size'] = 512
# adatas['human'].uns['chunk_size'] = 512
# adatas['macaque'].uns['chunk_size'] = 512
# adatas['mouse'].var["chunk_index"] = np.arange(adatas['mouse'].var.shape[0]) // 512
# adatas['human'].var["chunk_index"] = np.arange(adatas['human'].var.shape[0]) // 512
# adatas['macaque'].var["chunk_index"] = np.arange(adatas['macaque'].var.shape[0]) // 512


In [ ]:
for s in adatas.keys():
    crested.pp.train_val_test_split(
        adatas[s], strategy="region", val_size=0.1, test_size=0.1, random_state=42
    )


In [ ]:
meta_module = crandata._anndatamodule.MetaAnnDataModule(
    adatas=list(adatas.values()),
    genomes=list(genomes.values()),
    data_sources={'y': 'X'},
    in_memory=False,
    random_reverse_complement=True,
    max_stochastic_shift=10,
    deterministic_shift=False,
    shuffle_obs=False, obs_alignment = 'intersect',
    shuffle=True,
    batch_size=32,    # small batch size for testing
    epoch_size=1000000    # small epoch size for quick testing
)

# Setup the meta module for the "fit" stage (train/val)
meta_module.setup("fit")

# Retrieve the training dataloader from the meta module and iterate over a couple of batches.
meta_train_dl = meta_module.train_dataloader

print("\nIterating over a couple of training batches from MetaAnnDataModule:")
for i, batch in enumerate(tqdm(meta_train_dl.data)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.shape}")
    if i == 5:
        break


In [ ]:
for i, batch in enumerate(tqdm(meta_train_dl.data)):
    print(f"Meta Batch {i}:")
    for key, tensor in batch.items():
        print(f"  {key}: shape {tensor.dtype}")
    if i == 5:
        break


In [ ]:
import cProfile

code = '''
for i, batch in enumerate(meta_train_dl.data):
    # print(f"Meta Batch {i}:")
    # for key, tensor in batch.items():
    #     print(f"  {key}: shape {tensor.shape}")
    if i == 5:
        break
'''

out = cProfile.run(code,sort=True)


In [ ]:
model_architecture = crested.tl.zoo.simple_convnet(
    seq_len=2114, num_classes=batch['y'].shape[1]
)


In [ ]:
import keras
# Create your own configuration
# I recommend trying this for peak regression with a weighted cosine mse log loss function
optimizer = keras.optimizers.Adam(learning_rate=1e-5)
loss = crested.tl.losses.CosineMSELogLoss(max_weight=100, multiplier=1)
loss = crested.tl.losses.PoissonLoss()

metrics = [
    keras.metrics.MeanAbsoluteError(),
    # keras.metrics.MeanSquaredError(),
    # keras.metrics.CosineSimilarity(axis=1),
    crested.tl.metrics.PearsonCorrelation(),
    # crested.tl.metrics.ConcordanceCorrelationCoefficient(),
    # crested.tl.metrics.PearsonCorrelationLog(),
    # crested.tl.metrics.ZeroPenaltyMetric(),
]

alternative_config = crested.tl.TaskConfig(optimizer, loss, metrics)
print(alternative_config)


In [ ]:
# initialize some lazy model parameters *yawn*
model_architecture(batch)

In [ ]:
trainer = crested.tl.Crested(
    data=meta_module,
    model=model_architecture,
    config=alternative_config,
    project_name="mouse_biccn",  # change to your liking
    run_name="basemodel",  # change to your liking
    logger=None,  # or None, 'dvc', 'tensorboard'
    seed=7,  # For reproducibility
)
# train the model
trainer.fit(
    epochs=60,
    learning_rate_reduce_patience=3,
    early_stopping_patience=6,
)
